In [1]:
from google.colab import drive
import shutil

drive.mount('/content/drive', force_remount=True)
source_folder_path1 = '/content/drive/MyDrive/FaceAlignment'
destination_folder_path1 = '/content/FaceAlignment'
shutil.copytree(source_folder_path1, destination_folder_path1)

source_folder_path2 = '/content/drive/MyDrive/FaceAlignmentV2'
destination_folder_path2 = '/content/FaceAlignmentV2'
shutil.copytree(source_folder_path2, destination_folder_path2)

Mounted at /content/drive


'/content/FaceAlignmentV2'

In [7]:
import glob
import zipfile
import os

zip_file_path = f"/content/FaceAlignment/LFW/lfw_aligned_mtcnn.zip"
extracted_folder_path = f"/content/lfw_aligned_mtcnn"
os.makedirs(extracted_folder_path, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

**SDD-FIQA Codes**

In [3]:
!git clone https://github.com/Tencent/TFace.git

Cloning into 'TFace'...
remote: Enumerating objects: 848, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 848 (delta 36), reused 92 (delta 30), pack-reused 723 (from 1)
Receiving objects: 100% (848/848), 28.29 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [4]:
!mkdir /content/TFace/quality/model
!cp /content/FaceAlignment/SDD-FIQA/SDD_FIQA_checkpoints_r50.pth /content/TFace/quality/model/

In [5]:
%cd /content/TFace/quality

/content/TFace/quality


In [6]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [8]:
import os
import os.path as osp
import torch
import torch.nn as nn
import torch.optim as opti
from tqdm import tqdm
import torchvision.transforms as T
from generate_pseudo_labels.extract_embedding.model import model
import numpy as np
from scipy import stats
import pdb
from PIL import Image
import glob
import csv

def read_img(imgPath):     # read image & data pre-process
    data = torch.randn(1, 3, 112, 112)
    transform = T.Compose([
        T.Resize((112, 112)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    img = Image.open(imgPath).convert("RGB")
    data[0, :, :, :] = transform(img).to(torch.float32)
    return data


def network(eval_model, device):
    net = model.R50([112, 112], use_type="Qua").to(device)
    net_dict = net.state_dict()
    data_dict = {
        key.replace('module.', ''): value for key, value in torch.load(eval_model, map_location=device).items()}
    net_dict.update(data_dict)
    net.load_state_dict(net_dict)
    net.eval()
    return net

device = 'cuda:0'                                     # 'cpu' or 'cuda:x'
eval_model = './model/SDD_FIQA_checkpoints_r50.pth'   # checkpoint
net = network(eval_model, device)

csv_file = f"/content/lfw_mtcnn_aligned_sddfiqa.csv"

dataset_path = f'/content/lfw_aligned_mtcnn/*.jpg'
with open(csv_file,mode='w',newline='') as file:
  csv_writer = csv.writer(file)
  csv_writer.writerow(["Image","Score"])
  for index,img_path in enumerate(glob.glob(dataset_path)):
    input_data = read_img(img_path)
    pred_score = net(input_data.to(device)).data.cpu().numpy().squeeze() / 100
    csv_writer.writerow([img_path.split("/")[-1],pred_score])
    print(f'{index} -- {img_path} -- {pred_score}')

0 -- /content/lfw_aligned_mtcnn/Barry_Diller_0001.jpg -- 0.36922305822372437
1 -- /content/lfw_aligned_mtcnn/Ben_Braun_0001.jpg -- 0.49316585063934326
2 -- /content/lfw_aligned_mtcnn/Roman_Polanski_0005.jpg -- 0.4463679790496826
3 -- /content/lfw_aligned_mtcnn/Matt_Braker_0001.jpg -- 0.4665767252445221
4 -- /content/lfw_aligned_mtcnn/Edie_Falco_0002.jpg -- 0.4661715626716614
5 -- /content/lfw_aligned_mtcnn/Nia_Vardalos_0003.jpg -- 0.4304685592651367
6 -- /content/lfw_aligned_mtcnn/Nestor_Kirchner_0034.jpg -- 0.5326351523399353
7 -- /content/lfw_aligned_mtcnn/Susie_Castillo_0001.jpg -- 0.5298219919204712
8 -- /content/lfw_aligned_mtcnn/Nancy_Pelosi_0006.jpg -- 0.4156569540500641
9 -- /content/lfw_aligned_mtcnn/Fernando_Gonzalez_0004.jpg -- 0.41914302110671997
10 -- /content/lfw_aligned_mtcnn/Will_Ferrell_0001.jpg -- 0.3978792428970337
11 -- /content/lfw_aligned_mtcnn/Nestor_Kirchner_0003.jpg -- 0.4505557715892792
12 -- /content/lfw_aligned_mtcnn/Vladimir_Putin_0049.jpg -- 0.345316886901

KeyboardInterrupt: 

In [ ]:
import shutil

for model in ["mtcnn","rf"]:
  for op in ["cropped","aligned"]:
    csv_file = f"/content/ijbb_{model}_{op}_sddfiqa.csv"
    source_file = csv_file
    destination_directory = "/content/drive/MyDrive/FaceAlignmentV2/IJBB"
    shutil.copy(source_file, destination_directory)
    print(f"File copied from {source_file} to {destination_directory}")

File copied from /content/ijbb_mtcnn_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/IJBB
File copied from /content/ijbb_mtcnn_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/IJBB
File copied from /content/ijbb_rf_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/IJBB
File copied from /content/ijbb_rf_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/IJBB


In [ ]:
import glob
import zipfile
import os

for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{model}_{op}'
    zip_file_path = f"/content/FaceAlignmentV2/SCFace/{scface}.zip"
    extracted_folder_path = f"/content/{scface}"
    os.makedirs(extracted_folder_path, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)

In [ ]:
import os
import os.path as osp
import torch
import torch.nn as nn
import torch.optim as opti
from tqdm import tqdm
import torchvision.transforms as T
from generate_pseudo_labels.extract_embedding.model import model
import numpy as np
from scipy import stats
import pdb
from PIL import Image
import glob
import csv

def read_img(imgPath):     # read image & data pre-process
    data = torch.randn(1, 3, 112, 112)
    transform = T.Compose([
        T.Resize((112, 112)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    img = Image.open(imgPath).convert("RGB")
    data[0, :, :, :] = transform(img).to(torch.float32)
    return data


def network(eval_model, device):
    net = model.R50([112, 112], use_type="Qua").to(device)
    net_dict = net.state_dict()
    data_dict = {
        key.replace('module.', ''): value for key, value in torch.load(eval_model, map_location=device).items()}
    net_dict.update(data_dict)
    net.load_state_dict(net_dict)
    net.eval()
    return net

device = 'cuda:0'                                     # 'cpu' or 'cuda:x'
eval_model = './model/SDD_FIQA_checkpoints_r50.pth'   # checkpoint
net = network(eval_model, device)
for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{model}_{op}'
    csv_file = f"/content/{scface}_sddfiqa.csv"

    dataset_path = f'/content/{scface}/*.jpg'
    with open(csv_file,mode='w',newline='') as file:
      csv_writer = csv.writer(file)
      csv_writer.writerow(["Image","Score"])
      for index,img_path in enumerate(glob.glob(dataset_path)):
        input_data = read_img(img_path)
        pred_score = net(input_data.to(device)).data.cpu().numpy().squeeze() / 100
        csv_writer.writerow([img_path.split("/")[-1],pred_score])
        print(f'{index} -- {img_path} -- {pred_score}')

<ipython-input-14-c6c55bdb4525>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  key.replace('module.', ''): value for key, value in torch.load(eval_model, map_location=dev

Streaming output truncated to the last 5000 lines.
28 -- /content/scface_d2_c1_rf_cropped/047_cam1_2.jpg -- 0.31604522705078125
29 -- /content/scface_d2_c1_rf_cropped/016_cam1_2.jpg -- 0.2538283348083496
30 -- /content/scface_d2_c1_rf_cropped/100_cam1_2.jpg -- 0.36890579223632813
31 -- /content/scface_d2_c1_rf_cropped/120_cam1_2.jpg -- 0.3509500503540039
32 -- /content/scface_d2_c1_rf_cropped/112_cam1_2.jpg -- 0.27812704086303713
33 -- /content/scface_d2_c1_rf_cropped/073_cam1_2.jpg -- 0.32107326507568357
34 -- /content/scface_d2_c1_rf_cropped/075_cam1_2.jpg -- 0.3579755020141602
35 -- /content/scface_d2_c1_rf_cropped/080_cam1_2.jpg -- 0.29424270629882815
36 -- /content/scface_d2_c1_rf_cropped/122_cam1_2.jpg -- 0.4033343505859375
37 -- /content/scface_d2_c1_rf_cropped/069_cam1_2.jpg -- 0.3656869125366211
38 -- /content/scface_d2_c1_rf_cropped/109_cam1_2.jpg -- 0.3423151397705078
39 -- /content/scface_d2_c1_rf_cropped/008_cam1_2.jpg -- 0.4193821334838867
40 -- /content/scface_d2_c1_rf_c

In [ ]:
import shutil

for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{model}_{op}'
    csv_file = f"/content/{scface}_sddfiqa.csv"
    source_file = csv_file
    destination_directory = "/content/drive/MyDrive/FaceAlignmentV2/SCFace"
    shutil.copy(source_file, destination_directory)
    print(f"File copied from {source_file} to {destination_directory}")

File copied from /content/scface_d1_c1_mtcnn_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_rf_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_mtcnn_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_rf_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_mtcnn_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_rf_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_mtcnn_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_rf_aligned_sddfiqa.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c3_mtcnn_cropped_sddfiqa.csv to /content/drive/MyDrive/FaceAlign